In [4]:
# Импортируем библиотеки и токен
import requests
import matplotlib
import pandas as pd
import numpy as np
import datetime as dt
import time
from acces_token_webinar import m_web_tok_2

# Забираю Данные о мероприятиях

URL = 'https://userapi.webinar.ru/v3/organization/events/schedule'
headers = {'x-auth-token':m_web_tok_2,
            }
params = {'from':'2022-01-01'}
    
data_event = requests.get(url=URL, headers = headers, params=params)
#print(f"Status code: {data_event.status_code}")

response_dict = data_event.json()


keyses_d = []
for repo_dict in response_dict:
    keyses_d.append(repo_dict.keys())

response_dict

# Забираю ID тестов для дальнейшего построения цикла.
URL = 'https://userapi.webinar.ru/v3/tests/list'
headers = {'x-auth-token':m_web_tok_2,
            }
params = {'startTime':'2022-01-02 00:00:00'}
    
data_event = requests.get(url=URL, headers = headers, params=params)
#print(f"Status code: {data_event.status_code}")

response_dict_da_da = data_event.json()


# Цикл для забора всех ID-тестов
test_id = []
for repo_dict in response_dict_da_da:
    test_id.append(repo_dict['id'])

test_id

# Цикл для получения информации (Вопрос, ответ и т.п) по каждому ID-теста
all_tests = []
for id in test_id:
    URL_t = f'https://userapi.webinar.ru/v3/tests/{id}/results'
    headers_t = {'x-auth-token':m_web_tok_2,
                }
    params_t = {'startTime':'2023-07-02'}

    data_test = requests.get(url=URL_t, headers = headers_t)
    time.sleep(0.5)
    response_dict = data_test.json()
    if data_test.status_code == 200:
        all_tests.append(response_dict)
    else:
        continue

## Далее наму нужно:
# 1 - создать списки с набором данных
 
# 2 - Построить DataFrame из списков
 
# 3 - Достать информацию по вебинару с eventSessionId
 
# 4 - По ключу eventSessionId присоединить test_id



# 1 - создать списки с набором данных:
# Создаем списки через несколько циклов вложенных друг в друга.
# C id-теста, ответом, id-участника, вопросом
ids, answers, users_id_q, questions = [], [], [], []
for test in all_tests:
    for t in test:
        for ind in t['users']:
            for answ in ind['answers']:
                ids.append(t['id'])
                users_id_q.append(ind['id'])
                questions.append(answ['questionData'])
                answers.append(answ['answerData'])
#                names.append(ind['name'])

# 2 - Построить DataFrame из списков
data_df = []
data_df.append(ids)
data_df.append(users_id_q)
#data_df.append(names)
data_df.append(questions)
data_df.append(answers)

df_q = pd.DataFrame(data = data_df).transpose()
df_q.columns = ['test_id','users_id',  'question', 'answer']


df_q['test_id'] = df_q['test_id'].astype(str).astype('int64')

# Создаем списки из данных о вебинаре через вложенные циклы:
test_ids, event_ids, dates = [], [], []
for repo_dict in response_dict_da_da:
    for data in repo_dict['testSessions']:
        test_ids.append(data['id'])
        event_ids.append(data['eventSessionId'])
        dates.append(data['startTime'])
    

# Строим DF для id вебинаров
data_df_events = []
data_df_events.append(test_ids)
data_df_events.append(event_ids)
data_df_events.append(dates)


df_ev = pd.DataFrame(data = data_df_events).transpose()
df_ev.columns = ['test_id','event_ids', 'dates']


df_q_ev = df_q.merge(df_ev, how = 'left', on='test_id')



# 3 - Достать информацию по вебинару с eventSessionId, 
# Чтобы потом по eventSessionId добавить название мероприятия, дату и имя организатора.
# В основной дата-сет.
URL_ev = 'https://userapi.webinar.ru/v3/stats/events'
headers_ev = {'x-auth-token':m_web_tok_2,
            }
params_ev = {
    'from':'2023-07-01',
    'to':'2023-07-05',
}
    
event_info = requests.get(url=URL_ev, headers = headers_ev, params=params_ev)

# Форматируем полученный ответ в формат JSON
response_dict_ev_inf = event_info.json()

# Создаем списки с данными
creaters_id, cr_names, cr_second_names, ev_ids, ev_names, dates = [], [], [], [], [], []
for ev_inf in response_dict_ev_inf:
    creaters_id.append(ev_inf['createUser']['id'])
    cr_names.append(ev_inf['createUser']['name'])
    cr_second_names.append(ev_inf['createUser']['secondName']) 
    ev_ids.append(ev_inf['id'])
    ev_names.append(ev_inf['name'])
    dates.append(ev_inf['startsAt'])



# Строим DF с информацией по вебинарам.
# creaters_id, cr_names, cr_second_names, ev_ids, ev_names, dates
df_events_info = []
df_events_info.append(creaters_id)
df_events_info.append(cr_names)
df_events_info.append(cr_second_names)
df_events_info.append(ev_ids)
df_events_info.append(ev_names)
df_events_info.append(dates)



df_ev_inf = pd.DataFrame(data = df_events_info).transpose()
df_ev_inf.columns = ['creater_id','cr_name', 'cr_second_name', 'ev_id', 'ev_name', 'dates']

# Форматируем столбец с датой из формата - объект в формат - дата-время
df_ev_inf['dates'] = pd.to_datetime(df_ev_inf['dates'], utc = True)
# Убираем временную зону (она не поддерживается в excel).
df_ev_inf['dates'] = df_ev_inf['dates'].dt.tz_localize(None)

# Превращаем столбец с ID вебинара в числовой формат для дальнейшего объединения:
df_ev_inf['ev_id'] = df_ev_inf['ev_id'].astype(str).astype('int64')
df_ev_inf.rename(columns = {'ev_id':'event_ids'}, inplace = True)

# Объединяем дата-фреймы с данными о вебинаре и данными о тесте.
df_tests_info = df_q_ev.merge(df_ev_inf, how = 'left', on ='event_ids')

# Сортируем по дате теста
df_tests_info.sort_values(by = ['dates_x'], ascending = False, inplace =True)

# Форматируем столбец с датой из формата - объект в формат - дата-время
df_tests_info['dates_y'] = pd.to_datetime(df_tests_info['dates_y'])
df_tests_info['dates_x'] = pd.to_datetime(df_tests_info['dates_x'])

# Убераем временную зону:
df_tests_info['dates_y'] = df_tests_info['dates_y'].dt.tz_localize(None)
df_tests_info['dates_x'] = df_tests_info['dates_x'].dt.tz_localize(None)


df_tests_info.to_excel('all_data_tests_new_resp.xlsx', index = False)